# Multi-objective Decision Making

## Urban Transportation Planning

**Urban Transportation Planning Model**

In our urban transportation planning scenario, we aim to optimize the city's public transportation network by focusing on three main objectives: minimizing travel time, minimizing environmental impact, and minimizing costs. Here's how we can mathematically model these objectives and the constraints we face:

**Decision Variables:**
- $x_{ij}$: The number of transportation units (e.g., buses, trains) allocated to route $i$ during time period $j$.

**Parameters:**
- $t_{ij}$: Average travel time per unit on route $i$ during time period $j$.
- $e_{ij}$: Environmental impact per transportation unit on route $i$ during time period $j$, measured in terms of emissions.
- $c_{ij}$: Operational cost per transportation unit on route $i$ during time period $j$.
- $B$: Total budget available for transportation projects.
- $E_{max}$: Maximum allowable environmental impact.
- $D_{ij}$: Demand for transportation on route $i$ during time period $j$.

**Objective Functions:**
1. Minimize Total Travel Time:
   $$Z_1 = \sum_{i}\sum_{j} t_{ij} x_{ij}$$
2. Minimize Environmental Impact:
   $$Z_2 = \sum_{i}\sum_{j} e_{ij} x_{ij}$$
3. Minimize Costs:
   $$Z_3 = \sum_{i}\sum_{j} c_{ij} x_{ij}$$

**Constraints:**
1. Budget Constraint:
   $$\sum_{i}\sum_{j} c_{ij} x_{ij} \leq B$$
2. Environmental Impact Constraint:
   $$\sum_{i}\sum_{j} e_{ij} x_{ij} \leq E_{max}$$
3. Demand Satisfaction:
   $$x_{ij} \geq D_{ij}, \forall i, j$$
4. Non-negativity and Integer Constraints:
   $$x_{ij} \geq 0 \text{ and integer}, \forall i, j$$

In [1]:
from gurobipy import Model, GRB, quicksum

# Initialize the model
model = Model("UrbanTransportationPlanning")

# Sample data: Indices for routes (i) and time periods (j)
routes = ['Route1', 'Route2', 'Route3']  # Example routes
time_periods = ['Morning', 'Evening']  # Example time periods

# Parameters
t_ij = {(r, t): 10 + i + j for i, r in enumerate(routes) for j, t in enumerate(time_periods)}  # Travel time
e_ij = {(r, t): 5 + i + j for i, r in enumerate(routes) for j, t in enumerate(time_periods)}  # Environmental impact
c_ij = {(r, t): 100 + 10*i + 10*j for i, r in enumerate(routes) for j, t in enumerate(time_periods)}  # Cost
B = 10000  # Total budget
E_max = 300  # Max environmental impact
D_ij = {(r, t): 2 + i + j for i, r in enumerate(routes) for j, t in enumerate(time_periods)}  # Demand

# Decision variables: Number of transportation units allocated to route i during time period j
x_ij = model.addVars(routes, time_periods, vtype=GRB.INTEGER, name="x_ij")

# Objective functions
total_travel_time = quicksum(t_ij[r, t] * x_ij[r, t] for r in routes for t in time_periods)
total_environmental_impact = quicksum(e_ij[r, t] * x_ij[r, t] for r in routes for t in time_periods)
total_cost = quicksum(c_ij[r, t] * x_ij[r, t] for r in routes for t in time_periods)

# Set multi-objective
model.ModelSense = GRB.MINIMIZE
model.setObjectiveN(total_travel_time, index=0, priority=2, name="MinimizeTravelTime")
model.setObjectiveN(total_environmental_impact, index=1, priority=1, name="MinimizeEnvironmentalImpact")
model.setObjectiveN(total_cost, index=2, priority=1, name="MinimizeCost")

# Constraints
model.addConstr(total_cost <= B, "BudgetConstraint")
model.addConstr(total_environmental_impact <= E_max, "EnvironmentalImpactConstraint")
for r in routes:
    for t in time_periods:
        model.addConstr(x_ij[r, t] >= D_ij[r, t], f"Demand_{r}_{t}")

# Solve the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for r in routes:
        for t in time_periods:
            print(f" - Units allocated to {r} during {t}: {x_ij[r, t].X}")
    print(f"Objective Values: Travel Time = {total_travel_time.getValue()}, Environmental Impact = {total_environmental_impact.getValue()}, Cost = {total_cost.getValue()}")
else:
    print("No optimal solution found.")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 6 columns and 18 nonzeros
Model fingerprint: 0x022811a7
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+04]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 7 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 6 columns, 6 nonzer